<a href="https://colab.research.google.com/github/sabbas123/Comparing-two-videos-using-keypoints-using-Jaccard-Similarity/blob/main/ArSL_gloss_of_Friday_Sermon_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel('/content/drive/MyDrive/gloss notation with +.xlsx')

# Initialize sets to hold the unique phrases for each signer and both combined
unique_phrases = set()
unique_phrases_signer_1 = set()
unique_phrases_signer_2 = set()

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Ensure the row data is a string and split the string by '+' and strip whitespace
    phrases_signer_1 = [phrase.strip() for phrase in str(row['Gloss of signer 1']).split('+')]
    phrases_signer_2 = [phrase.strip() for phrase in str(row['Gloss of signer 2']).split('+')]

    # Add the phrases to their respective sets
    unique_phrases_signer_1.update(phrases_signer_1)
    unique_phrases_signer_2.update(phrases_signer_2)

    # Combine both sets for the total unique phrases
    unique_phrases.update(phrases_signer_1 + phrases_signer_2)

# Count the number of unique phrases for each signer and combined
total_unique_phrases = len(unique_phrases)
total_unique_phrases_signer_1 = len(unique_phrases_signer_1)
total_unique_phrases_signer_2 = len(unique_phrases_signer_2)

print(f"Total unique phrases/words gloss for signer 1 (without repetitions): {total_unique_phrases_signer_1}")
print(f"Total unique phrases/words gloss for signer 2 (without repetitions): {total_unique_phrases_signer_2}")
print(f"Total unique phrases/words gloss (without repetitions): {total_unique_phrases}")


Total unique phrases/words gloss for signer 1 (without repetitions): 192
Total unique phrases/words gloss for signer 2 (without repetitions): 187
Total unique phrases/words gloss (without repetitions): 201


In [ ]:
import os
import pandas as pd

def read_text_files(folder_path):
    data = []
    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                # Determine the signer based on the file name
                signer = 'signer 1' if '1' in file else 'signer 2' if '2' in file else 'unknown'

                file_path = os.path.join(subdir, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    for line in lines[1:]:  # Skip the first line
                        if line.strip() and not line.startswith("Begin Time"):  # Skip empty lines and lines containing column names
                            begin_time, end_time, duration, gloss = line.split('\t')
                            data.append({
                                'Sentences': os.path.basename(subdir),
                                'Signer': signer,
                                'Begin Time': begin_time,
                                'End Time': end_time,
                                'Duration': duration,
                                'GLOSS': gloss.strip()  # Remove whitespace and newlines
                            })
    return data

# Path to the folder on Google Drive
folder_path = '/content/drive/MyDrive/annotated ArSL data (gloss)'

# Reading data from files
data = read_text_files(folder_path)

# Converting data to a DataFrame
df = pd.DataFrame(data)

# Path to the CSV file where data will be saved
output_csv_path = '/content/drive/MyDrive/annotated_ArSL_data.csv'
df.to_csv(output_csv_path, index=False)

print("Data has been successfully saved in a CSV file.")


Data has been successfully saved in a CSV file.


In [ ]:
import pandas as pd
import plotly.express as px

# CSV path
csv_path = '/content/drive/MyDrive/annotated_ArSL_data.csv'

# read CSV file
df = pd.read_csv(csv_path)

# Filter data for each signer and count occurrences
count_gloss_signer_1 = df[df['Signer'] == 'signer 1'].groupby('GLOSS').size().reset_index(name='Count')
count_gloss_signer_2 = df[df['Signer'] == 'signer 2'].groupby('GLOSS').size().reset_index(name='Count')

# Sort the counts in descending order
count_gloss_signer_1_sorted = count_gloss_signer_1.sort_values(by='Count', ascending=False)
count_gloss_signer_2_sorted = count_gloss_signer_2.sort_values(by='Count', ascending=False)

# Select top 20 glosses for each signer
top_gloss_signer_1 = count_gloss_signer_1_sorted.head(20)
top_gloss_signer_2 = count_gloss_signer_2_sorted.head(20)

# Plot for signer 1
fig_signer_1 = px.bar(top_gloss_signer_1, x='GLOSS', y='Count',
             title='Top 20 Most Frequent GLOSS by Signer 1',
             labels={'GLOSS': 'Gloss', 'Count': 'Frequency'},
             template='plotly_white')

# Reverse the x-axis for Arabic texts to display correctly
fig_signer_1.update_layout(xaxis=dict(autorange="reversed"))

# Plot for signer 2
fig_signer_2 = px.bar(top_gloss_signer_2, x='GLOSS', y='Count',
             title='Top 20 Most Frequent GLOSS by Signer 2',
             labels={'GLOSS': 'Gloss', 'Count': 'Frequency'},
             template='plotly_white')

fig_signer_2.update_layout(xaxis=dict(autorange="reversed"))

# Show the plots
fig_signer_1.show()
fig_signer_2.show()


In [ ]:
import pandas as pd
import plotly.express as px

# CSV path
csv_path = '/content/drive/MyDrive/annotated_ArSL_data.csv'

# read CSV file
df = pd.read_csv(csv_path)

# Group data based on 'GLOSS' and count times
count_gloss = df.groupby('GLOSS').size().reset_index(name='Count')

# Arrange the number of times in descending order
count_gloss_sorted = count_gloss.sort_values(by='Count', ascending=False)

#Sellect top 20 gloss repeated by two signers
top_gloss = count_gloss_sorted.head(20)


fig = px.bar(top_gloss, x='GLOSS', y='Count',
             title='Top 20 Most Frequent GLOSS by Both Signers',
             labels={'GLOSS': 'Gloss', 'Count': ' Frequency'},
             template='plotly_white')

# عكس المحور x للنصوص العربية لتظهر بشكل صحيح
fig.update_layout(xaxis=dict(autorange="reversed"))

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Replace with the path to your actual CSV file
csv_path = '/content/drive/MyDrive/annotated_ArSL_data.csv'

# Read the CSV file
df = pd.read_csv(csv_path)

# Create a numerical index for each unique sentence in the 'Folder Name' column
sentence_indices = {name: idx for idx, name in enumerate(df['Sentences'].unique())}
df['Sentence Index'] = df['Sentences'].map(sentence_indices)

# Filter the dataframe for signer 1 and signer 2
df_signer_1 = df[df['Signer'] == 'signer 1']
df_signer_2 = df[df['Signer'] == 'signer 2']

# Group by 'Sentence Index' and count the number of 'GLOSS' entries for each group for both signers
gloss_count_per_sentence_index_signer_1 = df_signer_1.groupby('Sentence Index')['GLOSS'].count().reset_index(name='GlossCount')
gloss_count_per_sentence_index_signer_2 = df_signer_2.groupby('Sentence Index')['GLOSS'].count().reset_index(name='GlossCount')

# Sort by 'Sentence Index' to maintain the order
gloss_count_per_sentence_index_signer_1 = gloss_count_per_sentence_index_signer_1.sort_values('Sentence Index')
gloss_count_per_sentence_index_signer_2 = gloss_count_per_sentence_index_signer_2.sort_values('Sentence Index')

# Create the bar chart for signer 1
fig_signer_1 = px.bar(gloss_count_per_sentence_index_signer_1, x='Sentence Index', y='GlossCount',
                      title='Number of Glosses per Sentence for Signer 1',
                      labels={'Sentence Index': 'Sentence Index', 'GlossCount': 'Num. of Glosses in Sentences'},
                      template='plotly_white')

# Create the bar chart for signer 2
fig_signer_2 = px.bar(gloss_count_per_sentence_index_signer_2, x='Sentence Index', y='GlossCount',
                      title='Number of Glosses per Sentence for Signer 2',
                      labels={'Sentence Index': 'Sentence Index', 'GlossCount': 'Num. of Glosses in Sentences'},
                      template='plotly_white')

# Show the plot for signer 1
fig_signer_1.show()

# Show the plot for signer 2
fig_signer_2.show()


In [ ]:
import pandas as pd

# Replace with the path to your actual CSV file
csv_path = '/content/drive/MyDrive/annotated_ArSL_data.csv'

# Read the CSV file
df = pd.read_csv(csv_path)

# Filter the data for each signer
df_signer_1 = df[df['Signer'] == 'signer 1']
df_signer_2 = df[df['Signer'] == 'signer 2']

# Find the two sentences with the most glosses for each signer
top_two_signer_1 = df_signer_1.groupby('Sentences').size().nlargest(2).reset_index(name='Count')
top_two_signer_2 = df_signer_2.groupby('Sentences').size().nlargest(2).reset_index(name='Count')

# Print out the top two sentences for each signer
print("Top two sentences for Signer 1:")
print(top_two_signer_1)
print("\nTop two sentences for Signer 2:")
print(top_two_signer_2)


Top two sentences for Signer 1:
                            Sentences  Count
0  أجرنا من خزي الدنيا وعذاب الآخرة      8
1           أشهد ألا إله إلا الله      7

Top two sentences for Signer 2:
                            Sentences  Count
0  أجرنا من خزي الدنيا وعذاب الآخرة      7
1            صلى الله وسلم وبارك عليه      7


In [ ]:
import pandas as pd
import plotly.express as px



# Combine the data for Signer 1 and Signer 2 for plotting
combined_data = pd.concat([
    top_two_signer_1.assign(Signer='Signer 1'),
    top_two_signer_2.assign(Signer='Signer 2')
])

# Create a bar plot using Plotly Express
fig = px.bar(combined_data, x='Signer', y='Count', color='Sentences', barmode='group',
             title='Top Two Phrases with Most Glosses for Each Signer')


fig.show()
